In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import gensim as gs
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer, TFBertMainLayer, BertConfig

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU not found')
print('found GPU at {}'.format(device_name))

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
bert_model = TFBertModel.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
MAX_TOKENS = 250

In [ ]:
train_dataset = pd.read_csv('../../../train_data/train_data_T.csv', delimiter=',')
validation_dataset = pd.read_csv('../../../train_data/validation_data_T.csv', delimiter=',')

In [ ]:
# train_dataset = train_dataset.head(10)
# validation_dataset = validation_dataset.head(10)

In [ ]:
train_title_ids = []
train_title_mask = []
train_title_seg = []

train_label = []

for i,row in tqdm(train_dataset.iterrows()):
    
    article_title_main_passage = str(row['article_page_title'])+" "+str(row['article_meta_description'])
    table_title_main_passage = str(row['table_page_title'])+" "+str(row['table_page_summary'])
    
    return_tokenizer1 = bert_tokenizer.encode_plus(
      article_title_main_passage,
      table_title_main_passage,
      max_length=MAX_TOKENS,
      add_special_tokens=True,
      return_token_type_ids=True,
      pad_to_max_length=True,
      return_attention_mask=True,
    )
    
    train_title_ids.append(return_tokenizer1['input_ids'])
    train_title_mask.append(return_tokenizer1['attention_mask'])
    train_title_seg.append(return_tokenizer1['token_type_ids'])  
    
    train_label.append((row['label']))

In [ ]:
train_title_ids = np.array(train_title_ids)
train_title_mask = np.array(train_title_mask)
train_title_seg = np.array(train_title_seg)

train_label = np.array(train_label)

In [ ]:
validation_title_ids = []
validation_title_mask = []
validation_title_seg = []

validation_label = []

for i,row in tqdm(validation_dataset.iterrows()):
    
    article_title_main_passage = str(row['article_page_title'])+" "+str(row['article_meta_description'])
    table_title_main_passage = str(row['table_page_title'])+" "+str(row['table_page_summary'])
    
    return_tokenizer1 = bert_tokenizer.encode_plus(
      article_title_main_passage,
      table_title_main_passage,
      max_length=MAX_TOKENS,
      add_special_tokens=True,
      return_token_type_ids=True,
      pad_to_max_length=True,
      return_attention_mask=True,
    )
    
    validation_title_ids.append(return_tokenizer1['input_ids'])
    validation_title_mask.append(return_tokenizer1['attention_mask'])
    validation_title_seg.append(return_tokenizer1['token_type_ids'])  
    
    validation_label.append((row['label']))

In [ ]:
validation_title_ids = np.array(validation_title_ids)
validation_title_mask = np.array(validation_title_mask)
validation_title_seg = np.array(validation_title_seg)

validation_label = np.array(validation_label)

In [ ]:
# fast text embedding

In [ ]:
# embedding_model = gs.models.FastText.load('../../train_embedding_models/fasttext_embedding_50d_all_signals')
embedding_model = gs.models.FastText.load_fasttext_format('../../../pre_trained_models/cc.en.300.bin')

In [ ]:
MAX_PAD_TITLE = 30

def sequence_padding_title(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_TITLE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding_title(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_TITLE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding_title(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_TITLE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
MAX_PAD_MAIN_PASSAGE = 55

def sequence_padding_main_passage(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_MAIN_PASSAGE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding_main_passage(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_MAIN_PASSAGE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding_main_passage(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_MAIN_PASSAGE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
train_article_title = []
train_article_main_passage = []
train_article_keywords = []

train_table_title = []
train_table_main_passage = []
train_table_keywords = []

for i,row in tqdm(train_dataset.iterrows()):
    
    train_article_title.append(create_embedding_title(row['article_page_title']))
    train_table_title.append(create_embedding_title(row['table_page_title']))
    
    train_article_main_passage.append(create_embedding_main_passage(row['article_meta_description']))
    train_table_main_passage.append(create_embedding_main_passage(row['table_page_summary']))
    
    train_article_keywords.append(create_embedding_title(row['article_keywords']))
    train_table_keywords.append(create_embedding_title(row['table_page_keywords']))

In [ ]:
train_article_title = np.array(train_article_title, dtype='float16')
train_table_title = np.array(train_table_title, dtype='float16')
train_article_main_passage = np.array(train_article_main_passage, dtype='float16')
train_table_main_passage = np.array(train_table_main_passage, dtype='float16')
train_article_keywords = np.array(train_article_keywords, dtype='float16')
train_table_keywords = np.array(train_table_keywords, dtype='float16')

In [ ]:
validation_article_title = []
validation_article_main_passage = []
validation_article_keywords = []

validation_table_title = []
validation_table_main_passage = []
validation_table_keywords = []

for i, row in tqdm(validation_dataset.iterrows()):
    
    validation_article_title.append(create_embedding_title(row['article_page_title']))
    validation_table_title.append(create_embedding_title(row['table_page_title']))  
    
    validation_article_main_passage.append(create_embedding_main_passage(row['article_meta_description']))
    validation_table_main_passage.append(create_embedding_main_passage(row['table_page_summary']))
    
    validation_article_keywords.append(create_embedding_title(row['article_keywords']))
    validation_table_keywords.append(create_embedding_title(row['table_page_keywords']))

In [ ]:
validation_article_title = np.array(validation_article_title, dtype='float16')
validation_article_main_passage = np.array(validation_article_main_passage, dtype='float16')
validation_table_title = np.array(validation_table_title, dtype='float16')
validation_table_main_passage = np.array(validation_table_main_passage, dtype='float16')
validation_article_keywords = np.array(validation_article_keywords, dtype='float16')
validation_table_keywords = np.array(validation_table_keywords, dtype='float16')

In [ ]:
embedding_model = []

In [ ]:
title_ids = tf.keras.layers.Input(shape=(MAX_TOKENS,), name='input_ids', dtype='int32')
title_mask = tf.keras.layers.Input(shape=(MAX_TOKENS,), name='input_mask', dtype='int32')
title_seg = tf.keras.layers.Input(shape=(MAX_TOKENS,), name='input_seg', dtype='int32')

article_title = tf.keras.Input(shape=(MAX_PAD_TITLE,300), dtype='float32')
article_main_passage = tf.keras.Input(shape=(MAX_PAD_MAIN_PASSAGE,300), dtype='float32')
article_keywords = tf.keras.Input(shape=(MAX_PAD_TITLE,300), dtype='float32')

table_title = tf.keras.Input(shape=(MAX_PAD_TITLE,300), dtype='float32')
table_main_passage = tf.keras.Input(shape=(MAX_PAD_MAIN_PASSAGE,300), dtype='float32')
table_keywords = tf.keras.Input(shape=(MAX_PAD_TITLE,300), dtype='float32')

# article and tables signal representations
rep_article_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_title)
rep_article_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_main_passage)
rep_article_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_keywords)

rep_table_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_title)
rep_table_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_main_passage)
rep_table_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_keywords)

#bert vector
last_hidden_state, pooled_output = bert_model.bert([title_ids,title_mask,title_seg])

vector_A_title = tf.keras.layers.Flatten()(rep_article_title)
vector_A_Text = tf.keras.layers.Flatten()(rep_article_text)
vector_A_keywords = tf.keras.layers.Flatten()(rep_article_keywords)

vector_T_title = tf.keras.layers.Flatten()(rep_table_title)
vector_T_Text = tf.keras.layers.Flatten()(rep_table_text)
vector_T_keywords = tf.keras.layers.Flatten()(rep_table_keywords)


#learned vector
final_vector = tf.keras.layers.Concatenate()([pooled_output,vector_A_title,vector_A_Text,vector_A_keywords,vector_T_title,vector_T_Text,vector_T_keywords])
    
MLP_input = tf.keras.layers.Dense(512, activation='relu')(final_vector)
dropout1 = tf.keras.layers.Dropout(0.5, name="dropout1")(MLP_input)
MLP_hidden = tf.keras.layers.Dense(256, activation='relu')(dropout1)
dropout2 = tf.keras.layers.Dropout(0.5, name="dropout2")(MLP_hidden)
MLP_output = tf.keras.layers.Dense(1,activation='sigmoid')(dropout2)

model = tf.keras.Model(inputs=[title_ids,title_mask,title_seg, article_title, article_main_passage, article_keywords, table_title, table_main_passage, table_keywords],outputs=MLP_output)  

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

In [ ]:
model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
filepath="model_ablation05_{epoch:02d}_{val_accuracy:.4f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_format='tf', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(
    [train_title_ids, 
    train_title_mask, 
    train_title_seg,
    train_article_title,
    train_article_main_passage,
    train_article_keywords,
    train_table_title,
    train_table_main_passage,
    train_table_keywords], 
    train_label, 
    epochs=5, 
    batch_size=16,
    verbose=1,
    validation_data=(
    [validation_title_ids, 
    validation_title_mask, 
    validation_title_seg,
    validation_article_title,
    validation_article_main_passage,
    validation_article_keywords,
    validation_table_title,
    validation_table_main_passage,
    validation_table_keywords], 
    validation_label),
    callbacks=callbacks_list)